<a href="https://colab.research.google.com/github/shaya-686/AI/blob/main/%D0%94%D0%97_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [250]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/House_Rent_Dataset.csv", index_col="Unnamed: 0")

[Інформація про дані](https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset)

In [251]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100.0,Ground out of 2,NaN,NaN,Kolkata,Unfurnished,Bachelors/Family,2.0,Contact Owner
1,2022-05-13,2.0,20000.0,800.0,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1.0,NaN
2,2022-05-16,2.0,17000.0,1000.0,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1.0,Contact Owner
3,2022-07-04,2.0,10000.0,800.0,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1.0,Contact Owner
4,NaN,2.0,7500.0,850.0,1 out of 2,Carpet Area,NaN,Kolkata,Unfurnished,Bachelors,1.0,Contact Owner


In [252]:
df = df[['Rent', 'Size', 'Furnishing Status', 'City']]
df.head()

,Rent,Size,Furnishing Status,City
0,10000.0,1100.0,Unfurnished,Kolkata
1,20000.0,800.0,Semi-Furnished,Kolkata
2,17000.0,1000.0,Semi-Furnished,Kolkata
3,10000.0,800.0,Unfurnished,Kolkata
4,7500.0,850.0,Unfurnished,Kolkata


In [253]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4746 entries, 0 to 4745
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rent               4369 non-null   float64
 1   Size               4402 non-null   float64
 2   Furnishing Status  4395 non-null   object 
 3   City               4388 non-null   object 
dtypes: float64(2), object(2)
memory usage: 314.4+ KB


In [254]:
df.isna().sum()

Rent                 377
Size                 344
Furnishing Status    351
City                 358
dtype: int64

In [255]:
df = df.dropna(subset=['Rent'])

# Завдання 1
Ввиведіть основні числові характеристики даних, розділіть на тренувальну та тестову чатини

In [256]:
df.select_dtypes(include='number').describe()

,Rent,Size
count,4.369000e+03,4050.000000
mean,3.461306e+04,967.004444
std,7.740818e+04,630.252303
min,1.200000e+03,10.000000
25%,1.000000e+04,550.000000
50%,1.600000e+04,850.000000
75%,3.300000e+04,1200.000000
max,3.500000e+06,7000.000000


In [257]:
from sklearn.model_selection import train_test_split

y = df['Rent']
X = df.drop(columns='Rent')

X_original_train, X_original_test, y_original_train, y_original_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [258]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num_columns = X.select_dtypes(include="number").columns

num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
    )

# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [259]:
df.select_dtypes(include='object').describe()

,Furnishing Status,City
count,4049,4045
unique,3,6
top,Semi-Furnished,Mumbai
freq,1931,820


In [260]:
df['Furnishing Status'].value_counts()

Furnishing Status
Semi-Furnished    1931
Unfurnished       1530
Furnished          588
Name: count, dtype: int64

In [261]:
df['City'].value_counts()

City
Mumbai       820
Bangalore    757
Chennai      754
Hyderabad    738
Delhi        517
Kolkata      459
Name: count, dtype: int64

In [262]:
from sklearn.preprocessing import PowerTransformer
cat_columns = X.select_dtypes(include="object").columns
cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder())])

# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [263]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['Size'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index(['Furnishing Status', 'City'], dtype='object'))])

# Завдання 5
Створіть остаточну модель та натренуйте її

In [264]:
from sklearn.linear_model import LinearRegression

model = Pipeline(
    [("prep", preprocessor),
     ("model", LinearRegression())
     ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])

In [265]:
model.fit(X_original_train, y_original_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])

# Завдання 6
Виведіть основні метрики моделі

In [266]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

y_pred = model.predict(X_original_test)

all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_original_train, model.predict(X_original_train))
all_metrics["test"] = get_metrics(y_original_test, y_pred)


all_metrics

,train,test
MAE,24574.18,23456.96
RMSE,72071.11,42598.22
R2,0.23,0.36


# Завдання 7
Збережіть модель, завантажте її та спрогнозуйте ціну автомобіля

In [267]:
data = pd.DataFrame({'Size': [900],
                     'Furnishing Status': ['Semi-Furnished'],
                     'City': ['Mumbai']})

model.predict(data)

array([68737.7836765])

In [268]:
import joblib
joblib.dump(model, 'my_model.pkl')

['my_model.pkl']

In [269]:
load_model = joblib.load('my_model.pkl')
load_model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])